## Read Dataset

In [1]:
import numpy as np
import pandas as pd


data = pd.read_csv("EarthQuakes-Data-1970-2019.csv", sep=';')

## Show Dataset

In [2]:
data.columns

Index(['Date', 'Time', 'Latitude', 'Longitude', 'Depth', 'Magnitude'], dtype='object')

In [3]:
data.shape

(6063603, 6)

In [4]:
data.head()

,Date,Time,Latitude,Longitude,Depth,Magnitude
0,1/01/1970,01:19:30,15.90,-60.80,88.0,4.2
1,1/01/1970,01:36:17,-19.25,169.00,0.0,3.5
2,1/01/1970,01:43:48,8.54,-83.35,33.0,5.3
3,1/01/1970,01:49:56,28.53,129.39,49.0,5.1
4,1/01/1970,03:29:30,-4.90,102.83,170.0,4.9


In [5]:
data.tail()

,Date,Time,Latitude,Longitude,Depth,Magnitude
6063598,31/12/2019,23:53:22,45.07,-74.72,18.07,1.60
6063599,31/12/2019,23:53:33,18.00,-66.92,2.00,1.51
6063600,31/12/2019,23:54:01,17.92,-66.88,6.00,2.19
6063601,31/12/2019,23:56:59,-20.54,168.96,21.46,4.30
6063602,31/12/2019,23:59:40,17.86,-66.93,4.00,1.90


In [6]:
data.isnull().sum()

Date         0
Time         0
Latitude     0
Longitude    0
Depth        0
Magnitude    0
dtype: int64

In [7]:
data.describe()

,Latitude,Longitude,Depth,Magnitude
count,6.063603e+06,6.063603e+06,6.063603e+06,6.063603e+06
mean,2.988831e+01,5.323338e+01,2.947269e+01,2.125795e+00
std,2.238764e+01,1.016862e+02,5.819376e+01,1.250729e+00
min,-8.792000e+01,-1.800000e+02,-1.000000e+01,-2.700000e+00
25%,2.447000e+01,6.720000e+00,6.000000e+00,1.200000e+00
50%,3.585000e+01,1.205300e+02,1.100000e+01,2.000000e+00
75%,3.997000e+01,1.377700e+02,3.200000e+01,3.000000e+00
max,9.000000e+01,1.800000e+02,8.312000e+02,9.100000e+00


## Pre-Processing

### Convert time { 31/12/2019	23:53:22 } from type [datetime] to [float]

In [8]:
import datetime
import time

def date2float(d,t):
    ts = datetime.datetime.strptime(d+' '+t, '%d/%m/%Y %H:%M:%S')
    return time.mktime(ts.timetuple())


timestamp = []
for d, t in zip(data['Date'], data['Time']):
    try:
        _add = date2float(d,t)
        timestamp.append(_add)
    except ValueError:
        timestamp.append('ValueError')

timeStamp = pd.Series(timestamp)
data['Timestamp'] = timeStamp.values

In [9]:
final_data = data.drop(['Date', 'Time'], axis=1)
final_data = final_data[final_data.Timestamp != 'ValueError']

In [10]:
final_data.describe()

,Latitude,Longitude,Depth,Magnitude,Timestamp
count,6.063603e+06,6.063603e+06,6.063603e+06,6.063603e+06,6.063603e+06
mean,2.988831e+01,5.323338e+01,2.947269e+01,2.125795e+00,1.183859e+09
std,2.238764e+01,1.016862e+02,5.819376e+01,1.250729e+00,2.725687e+08
min,-8.792000e+01,-1.800000e+02,-1.000000e+01,-2.700000e+00,3.357000e+04
25%,2.447000e+01,6.720000e+00,6.000000e+00,1.200000e+00,1.055519e+09
50%,3.585000e+01,1.205300e+02,1.100000e+01,2.000000e+00,1.251181e+09
75%,3.997000e+01,1.377700e+02,3.200000e+01,3.000000e+00,1.370655e+09
max,9.000000e+01,1.800000e+02,8.312000e+02,9.100000e+00,1.577866e+09


In [11]:
final_data.head()

,Latitude,Longitude,Depth,Magnitude,Timestamp
0,15.90,-60.80,88.0,4.2,33570.0
1,-19.25,169.00,0.0,3.5,34577.0
2,8.54,-83.35,33.0,5.3,35028.0
3,28.53,129.39,49.0,5.1,35396.0
4,-4.90,102.83,170.0,4.9,41370.0


In [12]:
final_data.dtypes

Latitude     float64
Longitude    float64
Depth        float64
Magnitude    float64
Timestamp    float64
dtype: object

## Set up data train & data test

In [13]:
X = final_data[['Timestamp', 'Latitude','Longitude']]
y = final_data[['Depth', 'Magnitude']]

In [15]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,shuffle=True)
print(X_train.shape, X_test.shape, y_train.shape, X_test.shape)

(4850882, 3) (1212721, 3) (4850882, 2) (1212721, 3)


## Set up Model

In [19]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()
model.add(Dense(16, activation='relu', input_shape=(3,)))
model.add(Dense(16, activation='relu'))
model.add(Dense(2, activation='softmax'))

model.compile(optimizer='SGD', loss='squared_hinge', metrics=['accuracy'])

## train model

In [20]:
model.fit(X_train, y_train, batch_size=10, epochs=5, verbose=1, validation_data=(X_test, y_test))

Epoch 1/5
 74679/485089 [===>..........................] - ETA: 25:20 - loss: nan - accuracy: 0.8675

KeyboardInterrupt: 

## Evaluate

In [21]:
[test_loss, test_acc] = model.evaluate(X_test[:10], y_test[:10],verbose=0)
print("Evaluation result on Test Data : Loss = {}, accuracy = {}".format(test_loss, test_acc))

Evaluation result on Test Data : Loss = nan, accuracy = 0.800000011920929
